# OpenStreetMap Data Case Study

## Introduction

**Map Area:** Brooklyn (Kings County) in New York City, and surrounding neighborhoods including Southwest Queens and Far Rockaway, Lower Manhattan, and part of the New Jersey coastline bordering Manhattan.
I chose this area since it is where I reside.

[Link to extract](https://mapzen.com/data/metro-extracts/metro/brooklyn_new-york/)

**Problems in the Dataset**

After downloading the file in xml format, I ran procedures in the ElementTree module to find the following problems with the data set:

(1) Use of street abbreviations, for example "St." or "Ave" in place of "Street" or "Avenue".

(2) Inconsistent residential formatting: instances where an apartment unit number was included within the street address tag, rather than receiving its own entry/ key-value tag.

(3) Inconsistent commercial formatting: overlap and ambiguity when using the "amenity" or "shop" key for certain industries.

(4) Invalid formatting of house number tags. Some of these tags were not actually numbers but more general address specifications.

(5) Inconsistency in Zip code format (ex. "10002" vs "10002-1013" vs "NY 10002")

(6) City names either missing, inconsistent in content (ex. "Brooklyn, NY" vs "Brooklyn") or inconsistent in format. In the later case, an address in Kings Country, for example, could be tagged under "Brooklyn" or "New York City" depending upon the concept or popular usage of "city".

(7) Incorrect zip code and city formatting. This is an extension of (6). Since city tags were geographically inconsistent, the relation between zip codes and cities is also ambiguous. A zip code-city mapping should be implemented such that, whenever possible, each zip code refers to only one city.
After analyzing the data to find these problems, I wrote functions to clean the data in audit.py.

** SQL Database**

The cleaned data was then exported into a SQL database so it could be queried more efficiently. For example, the number of unique nodes, ways, and users:

In [2]:
cursor.execute(
'''SELECT COUNT(*)FROM nodes;''''
)
           cursor.execute ('''
           SELECT COUNT (*) FROM ways;'''
           )
           cursor.execute('''SELECT COUNT(DISTINCT(user.uid))
           FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as user;''')

SyntaxError: EOL while scanning string literal (<ipython-input-2-b873332aa1e1>, line 2)

We get 2,495,350 nodes; 492,780 ways, and 1,645 unique users.


## Problems in the Dataset

**1. Inconsistent Street Abbreviations**

By using regular expressions to parse through the data, I found the most common street abbreviations (those used more than twice) were:

'St': 104,<br>
'Ave': 26, <br>
'St.': 16,<br>
'Blvd': 12,


In addition, grammatical inconsistencies were present in non-abbreviated street types (ex. '70th Avenue,' , 'Bedford avenue') and in abbreviated street type ('South 4th st', 'Schermerhorn St.') Finally, there were a few instances of obvious misspellings such as 'Nostrand Avene' or 'West 8th Steet'.
To correct these tags, I compiled all abbreviations, grammatical inconsistencies, and spelling mistakes into a dictionary and mapped each key to its corrected form. This dictionary, named "streetmapping" is detailed below:
 

In [6]:
streetmapping = {
"Ave" : "Avenue",
"Ave." : "Avenue",
"Avene" : "Avenue",
"Avenue," : "Avenue",
"avenue" : "Avenue",
"ave": "Avenue",
"brway":"Broadway",
"Blvd" :"Boulevard",
"Ctr": "Center",
"Dr":"Drive",
"Plz" : "Plaza",
"Rd" : "Road",
"ST" : "Street",
'St': "Street",
'St.' : "Street",
 "St.," : "Street",
'Steet' : "Street",
'st' : "Street",
'street' : "Street",
"Street,":"Street"
}

**2. Unit or Apartment Address Included Within Street Address**

In most of the dataset, unit/apartment numbers are tagged separately. In some instances, however, these designations are appended to the street address. For example, the code bolded below:

{'website': 'http://adlervermillion.com (http://adlervermillion.com)', 'payment:bitcoin': 'yes', 'name': 'Adler, Vermillion & Skocilich LLP', 'addr:postcode': '11201', 'addr:housename': 'Adler, Vermillion & Skocilich LLP', 'addr:city': 'Brooklyn', 'addr:housenumber': '45', **'addr:street': 'Main St., Suite 500'**}


Addresses formatted like the above do not recognize the integrity of each piece of information specifying a given address. In the above example, just as house number 45 receives its own key-value tag, so should "Suite 500".


To find these discrepancies, I again used regular expressions to search street names ending in numbers. I compiled all the element tags of these street names into a dictionary to get a closer look.


In [ ]:
import xml.etree.cElementTree as ET
import re

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def audit (osmfile):
'''Searches for elements with streetnames that end in a string containing a number,
    and stores all tags for these elements in a list.'''

    osm_file = open(osmfile, "r")
    problemlist = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way": 
            for tag in elem.iter("tag"):
                if tag.attrib["k"] == "addr:street":
                    street_name = tag.attrib["v"]
                    street_endings = street_type_re.search(street_name)
                    street_endings = street_endings.group() 
                    numerical_street_endings = re.search(r'\d', street_endings)
                    
                    if numerical_street_endings:
                        numberstreets = {}
                        for tag in elem.iter("tag"):
                            numberstreets[tag.attrib["k"]] = tag.attrib["v"]
                        problemlist.append(numberstreets)   
                            
                           
    return problemlist

In all these cases, where a unit or apartment number was given within the street address, this number was listed immediately after the street type or street type abbreviation already stored within the streetmapping dictionary created above in (1). I was therefore able to use the occurrence of the street type (ex. "St.") to split the string into a substring specifying the street address ("Main St.") and a substring specifying the unit ("Suite 500"). I could then create a new tag ("Unit":"Suite 500") to store this information separately from the address. (Of course, in this case "Main St" would also need to be amended to "Main Street" as detailed in (1) above).

**3 Commercial Tagging**

In the Openstreetmap project, business are tagged under the "shop" or "amenity" key. However, because this schema is somewhat ambiguous, certain services/products are tagged under both types of keys depending upon the user. These overlapping cases are detailed below:

'funeral_directors':{'amenity tags': 1, 'shops tags': 1}, <br>
'laundry': {'amenity tags': 1, 'shops tags': 46}, <br>
'ice_cream': {'amenity tags': 9, 'shops tags': 2}

I corrected for this by amending "laundry" and "ice_cream" to be tagged under the shop key. In general, the "shop" tag seems to be a narrower or more specific concept better suited for these types of businesses. The amenity tag is broader category that includes more generally useful places such as "parking space" "marketspace" and "pharmacy".

However, because this schema is quite ambiguous, I would recommend replacing it with a more precise schema employing the categories "commercial" or "public" rather than the "shop" or "amenity" designation.

For example, once the data was clearned and exported to an SQL data base, I ran the following query in the terminal to find the most commonly tagged amenities:

In [ ]:
cursor.execute ('''SELECT COUNT(*) as count
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as tags
WHERE tags.key = "amenity";''')

supermarket,185<br>
clothes,156<br>
convenience,115 <br>
bakery,58 <br>
hairdresser,57

However while this public/private schema is a more precise and perhaps more intutive way of classifying addresses, it may also introduce new ambiguities into the dataset.

For example, depending upon the user, a non-profift facility such as a YMCA gym might be classified as either "public" or "commercial." The same problems pertain to such non-profit ventures such as Planned Parenthood and Goodwill, or even entire institutiosn such as hospitals or hospices.

Lastly, there were also some "shop" tags missing names. Running the query below for the total number of shops returns 1375

In [ ]:
 cursor.execute ('''SELECT COUNT(*) as count
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) as tag s
WHERE tags.key = "shop";''')

But performing an inner join and running a query for the total number of shops that also have a "name" value returns 974 and 253 respectively for nodes and ways. The percentage then of shops missing a name is about 11%

In [ ]:
cursor.execute ('''SELECT COUNT (*) as count
FROM nodes_tags as n1, nodes_tags as n2
WHERE (n1.key = "name" and n2.key = "shop") and n1.id = n2.id;''')
#Returns 974

cursor.execute ('''SELECT COUNT (*) as count
FROM ways_tags as w1, ways_tags as w2
WHERE (w1.key = "name" and w2.key = "shop") and w1.id = w2.id;''')
#Returns 253

**4. Housenumbers**

The code below runs a check that each value under "addr:housenumber" does contains a number of some sort, for example ("5B" or "5 1/2" would be permissible). Like before, the tags for elements failing this test are are compiled within a dictionary.

In [9]:
import xml.etree.cElementTree as ET

def parsehousenumbers (osmfile):
    problemlist = []
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way": 
            for tag in elem.iter("tag"):
                if tag.attrib["k"] == "addr:housenumber": housenumber = tag.attrib["v"]
                if not is_number (housenumber):
                    nonnumerical = {}
                    for tag in elem.iter("tag"):
                        nonnumerical[tag.attrib["k"]] = tag.attrib ["v"]
                    problemlist.append(nonnumerical)
    return problemlist

def is_number(s):
    '''http://stackoverflow.com/questions/31083503/how-do-i-check-if-a-s
           tring-contains-any-numbers'''
    if any(str.isdigit(c) for c in s): 
        return True
    else:
        return False


In cases, where an element failed this test, the housenumber tag was used more generally as an address specification. For example, the code below. In these cases, I corrected the "addr:housenumber" tag to a more general "addr:note" key

In [11]:
example =[
{'addr:city': 'Forest Hills',
  'addr:housenumber': 'South side of Queens Blvd at',
  'addr:postcode': '11375',
  'addr:street': '70th Avenue,',
  'area': 'yes',
  'name': 'Forest Hills Greenmarket',
  'note': '-Open Sundays, Year Round :) -Market Hours: 8:00 a.m. - 3:00 p.m. -Compost Collection w/ BIG!\
  Compost:10am-12pm -EBT/Food Stamps, Debit/Credit, and WIC & FMNP Coupons accepted 8:30am-2pm',
  'website': 'http://www.grownyc.org/greenmarket/queens/forest-hills'},
 {'addr:city': 'Forest Hills',
  'addr:housenumber': 'Deepdene Road and Underwood Road',
  'leisure': 'park',
  'name': 'Deepdene Park',
  'note': 'Great sledding in the WInter & firefly catching in warmer months! :)'
 }
]

**5. Zipcodes**

Zipcodes were formatted inconsistently throughout the dataset which I discovered running the code below:

In [13]:
import xml.etree.cElementTree as ET 
from collections import defaultdict

def audit (osmfile):
'''Returns a dictionary of zipcodes used within the dataset and the number of times each zipcode is used.'''
    zipcodedict = defaultdict(int)
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib["k"] == "addr:postcode":
                    zipcodedict[tag.attrib["v"]] +=1
    return zipcodedict

SyntaxError: invalid syntax (<ipython-input-13-c80b0e847d72>, line 1)

The majority of zip codes were formatted using a 5 number only system. Exceptions, for example, were 'NY 11201' (used twice) and '11201-2483' (used just once) to '11201' which was used in 3,895 tags. I corrected the data by using regular expressions to delete non-numerical characters and then splicing the string to get only the first five numbers. The third line of the string replaces zip code "10048" with "10007". The "10048" zip code was assigned to the original World Trade Center complex, and discontinued following the September 11 attacks in 2001.

In [14]:
import re

def correctzip (s):
    s = re.sub("\D", "", s) 
    s = s[0:5]
    if s == "10048":
        s = "10007"
    return s

**6. Cities**
Cities are used inconsistently throughout the dataset. For example, running the code below returns instances of all of the following number of tags under "addr:city":
            

In [16]:
{'Brookklyn': 20,
 'Brooklyn': 1747,
 'Brooklyn ': 2,
 'Brooklyn, NY': 5,
 'Brooklyn, New York': 1,
 'brooklyn': 7}

{'Brookklyn': 20,
 'Brooklyn': 1747,
 'Brooklyn ': 2,
 'Brooklyn, NY': 5,
 'Brooklyn, New York': 1,
 'brooklyn': 7}

In [17]:
def citiescount (osmfile):
    citydict = defaultdict(int)
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib["k"] == "addr:city":
                    citydict[tag.attrib["v"]] +=1
    return citydict

A full mapping of the code to correct all grammatical inconsistencies in the dataset is below:

In [18]:
city_mapping = {
"Brookklyn": "Brooklyn",
"brooklyn": "Brooklyn",
"Brooklyn, NY": "Brooklyn",
"Brooklyn, New York": "Brooklyn",
"Brooklyn ": "Brooklyn",
 "M": "New York",
"Manhattan NYC": "New York",
"NEW YORK CITY": "New York",
"New York CIty": "New York",
"New York City": "New York",
"New York, NY": "New York",
"Tribeca": "New York",
"York City": "New York",
"new york": "New York",
"Glendale, NY":"Glendale",
"Ozone Park, NY": "Ozone Park",
"queens": "Queens"
}

More interesting, however, than the misspellings or inconsistencies within a single city name such as "Brooklyn" is that the very category of a city is applied differently throughout the data. An address in Kings County, for example, was sometimes tagged under the city of "Brooklyn" - referring to its autonomous borough status, and sometimes tagged as "New York" - referring to its place within the five borough city. Similarly, an address in Queens County might be tagged as within the city of "Queens", as within the city of "New York", or most commonly as within the "city" of a neighborhood such as "Flushing" or "Elmhurst".


**7. Zipcode to City Mapping**

I explored these inconsistencies further by mapping out what city tags were matched to each zip code throughout the data. Before mapping cities to zip codes, I also cleaned the data for the inconsistencies previously noted. This way, I was able to explore what concepts of city were used throughout the data while also abstracting out the grammatical and stylistic inconsistencies in (5) and (6).

In [ ]:
def pullcitiesforzip (osmfile):
    cityzipmapping = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way": for tag in elem.iter("tag"):
            if tag.attrib["k"] == "addr:postcode": 
                zipcode = tag.attrib["v"]
                cityzipmapping[zipcode]
                
                for tag in elem.iter("tag"):
                    if tag.attrib["k"] == "addr:city":
                        city = tag.attrib["v"]
                        city = cleancity(city)
                        cityzipmapping[zipcode].add(city)
                        
    cityzipmapping = cleanzip(cityzipmapping)
    return cityzipmapping

def cleanzip (dictionary):
    cleanzip = {}
    for k,v in dictionary.items(): 
        newkey = correctzip(k)
        if newkey in cleanzip:
            if list (v) and list(v).pop() not in cleanzip[newkey]: 
                cleanzip[newkey].append(list(v))
        else:
            cleanzip[newkey] = (list(v))
    return cleanzip

def cleancity (city):
    if city in citymapping:
        city = citymapping[city]
    return city

The summarized results were as follows:

 - Following convention, Manhattan zip codes are listed under the city of "New York", most Brooklyn zip codes are listed under city the of "Brooklyn", most Queens zip codes are listed as cities under their specific neighborhood (ex. "Elmhurst", "Forest Hills" "Woodside" , etc).
 
However:

- Eleven zip codes ('10275', '10281', '11104', '11228', '11251', '11367', '11415', '11418', '11419', '11420', '11697') in the data have no city tagged anywhere throughout the data.
- Ten zip codes within Brooklyn ('11201', '11206', '11211', "11215, '11216', '11219', '11233', '11237', '11238', '11249') also contain elements in which "New York" or some variation thereof is tagged as the city.
- One Brooklyn zip code ('11203') also contains 'Waterbury' as its city. The closest "Waterbury" is in Connecticut so this tag is an obvious error.
- One zip code ('71877') tagged as within "Brooklyn" is actually a phone number.
- Five zip codes within Queens ('11373', '11374', '11375', '11377', '11385') also contain elements in which the city is given as either "Queens", "New York" or some variation therefore rather than the specific neighborhood.
- Of these Five, two zip codes in Queens ('11374', '11385') are also respectively listed as within the city of 'Rego Park'/ 'Forest Hills' / 'Flushing' and 'Glendale'/'Ridgewood' depending upon the user.


Drawing from these results, I created a new zip-code-to-city mapping that can be used to standardize this relationship within the data. This was done through the following steps
- Drawing upon [zipcodestogo](http://www.zipcodestogo.com/New%20York/) as a resource, I updated the mapping to include cities for the eleven zip codes missing a value within the osm file .
- Deleted 'New York' as a city from the 10 Brooklyn zip codes where this was included.
- Change zip code '11215' to refer to 'Brooklyn' rather than "New York.
- Deleted 'Waterbury' from the '11203' zip code
- Deleted "Queens" and "New York" from the 5 zip codes in Queens County which included a city tagged under this name.
- Deleted "Forest Hills" and "Flushing" as cities from 11374. Only "Rego Park" should be included within this zip code
[source](http://www.zipmap.net/zips/11374.htm)
- After researching the [controversy](https://www.dnainfo.com/new-york/20141016/glendale/glendale-wants-its-own-zip-code-break-away-from-ridgewood) regarding the '11385' zip code, I choose to admit an exception within the data so that the city in this zip code can be either "Ridgewood", "Glendale", or be left empty.


Using this zip-code-to-city mapping, I was also able to update all elements in the data that include a zip code but are missing a city. By running the code below I was able to determine there were 377,092 examples of missing cities that could be updated through this procedure. (Versus only 3,110 elements that have both city and zip code values).

In [20]:
def audit (osmfile):
    osm_file = open(osmfile, "r")
    count_city_listed = 0
    count_city_not_listed = 0
    other = 0
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            taglist = []
            for tag in elem.iter("tag"):
                taglist.append(tag.attrib["k"])
            if "addr:postcode" in taglist and "addr:city" not in taglist:
                count_city_not_listed += 1
            elif "addr:postcode" in taglist and "addr:city" in taglist:
                count_city_listed += 1 
            else:
                other +=1
                
    return (count_city_listed, count_city_not_listed, other)

After updating the file for these missing cities, we can run a search in the SQL database for the top five cities:

In [21]:
cursor.execute ('''SELECT tags.value, COUNT(*) as count
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags WHERE tags.key = "city"
GROUP BY tags.value
ORDER BY count DESC;''')

NameError: name 'cursor' is not defined

Brooklyn,292375 <br>
New York,17129 <br>
Ozone Park,9708 <br>
Middle Village,8714 <br>
Maspeth,8137 <br>
Forest Hills,77<br>

## Conclusions and Additional Thoughts

The audit.py file is largely capable of cleaning the data for the inconsistencies in content and format noted in points 1 - 7. Perhaps most significantly, a zip code to city mapping was extracted from within the data, audited, and then applied back to the file to establish a more uniform relation between these values. When there are not widespread errors in the data, this methodology is perhaps the best suited for insuring the internal validity of a dataset.


The limitation of this approach however is that it only tackles the relationships between the data points themselves, and cannot audit the relationship between the data and the "outside" world that such data is supposed to "represent". For datasets like the Openstreetmap Project, external accuracy, it seems to me, is largely dependent upon user-input and cannot be guaranteed by any data analysis alone. The role of the data analyst instead, might be to suggest where to begin looking for gaps between the data and the world.


For example, many of the commercial tags are very outdated for a city that so rapidly developing as Brooklyn. A cursory query of the timestamps on "restaurant" tags returns the following results.

In [ ]:
SELECT nodes.timestamp FROM nodes JOIN nodes_tags ON nodes.id = nodes_ta
gs.id WHERE nodes_tags.value = "restaurant" Limit 10;
2014-10-17T07:11:55Z
2009-07-25T20:35:59Z
2016-04-18T14:12:28Z
2012-06-28T18:29:26Z
2010-01-21T16:32:22Z
2010-01-21T16:32:21Z
2010-01-21T16:32:21Z
2010-01-21T16:32:21Z
2010-01-21T16:36:14Z
2010-01-21T16:36:14Z

My suggestion for the future of the Openstreetmap Project is for data analysts to locate commercial blocks with timestamps that are, on average, over a certain number of years. Such blocks could be labeled as areas the most needing of user input. Since New Yorkers are generally excited to explore new commercial areas of the city, this classification might hopefully incentivize further user feedback.
Although this approach respects the crowd-sourcing ethos of the Openstreetmap Project, one problem is that without a significant degree of user-diversification and participation, this commercial tagging project might not be inclusive of all areas of the city. In such case, requesting participation from governmental organizations such as NYC Department of Buildings or Department of Finance might be a more effective way of retrieving commercial data.